<a href="https://colab.research.google.com/github/LSoGon/Projeto_Imersao_IA/blob/main/Ferramenta_de_an%C3%A1lise_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Ferramenta de análise de Texto

Este código oferece uma maneira fácil e interativa de resumir textos extensos, extrair palavras-chave importantes, identificar descobertas e hipóteses, além de visualizar as relações entre conceitos em forma de mapa mental. Com apenas alguns cliques, os usuários podem obter uma visão concisa e organizada do conteúdo de um texto, economizando tempo e esforço na análise manual. Além disso, a integração com a biblioteca Gemini do Google oferece uma fonte confiável para a geração de conteúdo, garantindo resumos precisos e relevantes.

In [8]:
!pip install -q -U google-generativeai

In [2]:
# Importando a bibliotecas
import google.generativeai as genai
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
import networkx as nx
import re
from collections import Counter
from nltk.corpus import stopwords

In [10]:
# Configurando a API Key
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

# Inicializa o modelo generativo 'gemini-pro'
model = genai.GenerativeModel('gemini-pro')

In [4]:
# definindo função
def pesquisar_e_resumir(texto, formato_saida="texto"):
    """
    Pesquisa e resume artigos usando o Gemini.
    Args:
        texto: O texto a ser analisado.
        formato_saida: O formato de saída desejado (texto, gráfico ou mapa_mental).
    Returns:
        A saída formatada de acordo com o formato_saida.
    """
    # Define o prompt para o modelo Gemini
    prompt = f"""
    Você é um assistente de pesquisa. Analise o texto e forneça:
    - Resumo
    - Palavras-chave
    - Descobertas
    - Hipóteses
    - Mapa Mental (Conceitos e Relações)

    Formate a saída como:

    ## Resumo:
    [resumo]

    ## Palavras-chave:
    [palavra-chave 1], [palavra-chave 2], etc.

    ## Descobertas:
    - [descoberta 1]
    - [descoberta 2]

    ## Hipóteses:
    - [hipótese 1]
    - [hipótese 2]

    ## Mapa Mental (Conceitos):
    - [conceito 1]
    - [conceito 2]

    ## Mapa Mental (Relações):
    - [conceito 1] -- [relação] --> [conceito 2]

    Texto: {texto}
    """

    # Gera conteúdo com base no prompt usando o modelo Gemini
    resposta = model.generate_content(prompt)

    # Se o formato de saída desejado for texto, retorna o texto gerado pelo modelo
    if formato_saida == "texto":
        return resposta.text

    # Extrai informações da resposta usando expressões regulares
    resumo = re.search(r"## Resumo:\n(.*?)\n\n", resposta.text, re.DOTALL).group(1).strip() # Extrai o resumo
    palavras_chave = re.findall(r'\b\w+\b', re.findall(r"## Palavras-chave:\n(.*?)\n\n", resposta.text, re.DOTALL)[0]) # Extrai as palavras-chave
    descobertas = re.findall(r"- (.*?)\n", resposta.text, re.DOTALL) # Extrai as descobertas
    hipoteses = re.findall(r"- (.*?)\n", resposta.text, re.DOTALL) # Extrai as hipóteses
    conceitos = re.findall(r"- (.*?)\n", resposta.text, re.DOTALL) # Extrai os conceitos
    relacoes_raw = re.findall(r"- (.*?) -- (.*?) --> (.*?)\n", resposta.text, re.DOTALL) # Extrai as relações
    relacoes = [(r[0].strip(), r[2].strip(), r[1].strip()) for r in relacoes_raw] # Formata as relações em tuplas

    # Se o formato de saída for "gráfico", gera um gráfico de barras das palavras-chave mais frequentes
    if formato_saida == "gráfico":
        stop_words = set(stopwords.words('portuguese') + stopwords.words('english')) # Define a lista de stop words
        palavras = re.findall(r'\b\w+\b', texto.lower()) # Extrai todas as palavras do texto
        palavras_filtradas = [palavra for palavra in palavras if palavra not in stop_words] # Remove as stop words da lista de palavras
        contagem_palavras = Counter(palavras_filtradas) # Conta a frequência de cada palavra
        palavras_top_10, frequencias_top_10 = zip(*contagem_palavras.most_common(10)) # Obtém as 10 palavras mais frequentes e suas frequências

        # Cria o gráfico de barras
        plt.figure(figsize=(10, 5)) # Define o tamanho da figura
        plt.bar(palavras_top_10, frequencias_top_10) # Cria o gráfico de barras
        plt.xlabel("Palavras-chave") # Define o label do eixo x
        plt.ylabel("Frequência") # Define o label do eixo y
        plt.title("Top 10 Palavras-chave Mais Citadas") # Define o título do gráfico
        plt.xticks(rotation=45) # Rotaciona os labels do eixo x em 45 graus
        plt.tight_layout() # Ajusta o layout para evitar sobreposições
        plt.show() # Exibe o gráfico

    # Se o formato de saída for "mapa_mental", gera um mapa mental das relações entre os conceitos
    elif formato_saida == "mapa_mental":
        grafo = nx.Graph() # Cria um grafo vazio

        # Adiciona palavras-chave como nós no grafo
        for palavra_chave in palavras_chave:
            grafo.add_node(palavra_chave, label=palavra_chave) # Adiciona um nó para cada palavra-chave

        # Adiciona conceitos encurtados como nós no grafo
        for sentenca in conceitos:
            label_conceito = ' '.join(sentenca.split()[:3]) # Encurta o conceito para as 3 primeiras palavras
            if not grafo.has_node(label_conceito): # Se o conceito encurtado ainda não existir no grafo
                grafo.add_node(label_conceito, label=label_conceito) # Adiciona o conceito encurtado como um nó
            for palavra_chave in palavras_chave:
                if palavra_chave.lower() in sentenca.lower(): # Se a palavra-chave estiver presente no conceito
                    grafo.add_edge(palavra_chave, label_conceito) # Cria uma aresta entre a palavra-chave e o conceito

        # Adiciona relações encurtadas como arestas no grafo
        for conceito1, conceito2, relacao_str in relacoes:
            conceito1 = ' '.join(conceito1.split()[:3]) # Encurta o conceito 1 para as 3 primeiras palavras
            conceito2 = ' '.join(conceito2.split()[:3]) # Encurta o conceito 2 para as 3 primeiras palavras
            if not grafo.has_node(conceito1): # Se o conceito 1 ainda não existir no grafo
                grafo.add_node(conceito1, label=conceito1) # Adiciona o conceito 1 como um nó
            if not grafo.has_node(conceito2): # Se o conceito 2 ainda não existir no grafo
                grafo.add_node(conceito2, label=conceito2) # Adiciona o conceito 2 como um nó
            grafo.add_edge(conceito1, conceito2, label=relacao_str) # Cria uma aresta entre os conceitos 1 e 2 com o label da relação

        # Conecta todos os nós no grafo
        nos_visitados = set() # Cria um conjunto para armazenar os nós visitados
        fila = list(palavras_chave) # Cria uma fila para percorrer o grafo, começando pelas palavras-chave
        while fila: # Enquanto a fila não estiver vazia
            no_atual = fila.pop(0) # Remove o primeiro nó da fila
            if no_atual not in nos_visitados: # Se o nó ainda não tiver sido visitado
                nos_visitados.add(no_atual) # Marca o nó como visitado
                vizinhos = list(grafo.neighbors(no_atual)) # Obtém os vizinhos do nó atual
                fila.extend(vizinhos) # Adiciona os vizinhos à fila
        for no in grafo.nodes: # Para cada nó no grafo
            if no not in nos_visitados: # Se o nó não tiver sido visitado (ou seja, está isolado)
                grafo.add_edge(palavras_chave[0], no) # Conecta o nó à primeira palavra-chave

        # Desenha o mapa mental
        pos = nx.spring_layout(grafo)
        plt.figure(figsize=(16, 12))
        nx.draw(grafo, pos, with_labels=True, # Desenha o grafo com labels
                labels={node: grafo.nodes[node]['label'] for node in grafo.nodes()}, # Define os labels dos nós
                node_size=3000, node_color='lightblue', font_size=10, font_weight='bold', edge_color='gray') # Define as propriedades visuais dos nós e arestas
        edge_labels = nx.get_edge_attributes(grafo, 'label') # Obtém os labels das arestas
        nx.draw_networkx_edge_labels(grafo, pos, edge_labels=edge_labels, font_size=8) # Desenha os labels das arestas
        plt.title("Mapa Mental")
        plt.show()

    # Se o formato de saída for inválido, retorna uma mensagem de erro
    else:
        return "Formato de saída inválido."

In [5]:
# Cria uma interface do usuário
texto_input = widgets.Textarea(
    placeholder='Insira seu texto aqui',
    layout={'height': '400px', 'width': '90%', 'overflow': 'auto'}
)

# Cria um menu dropdown para o usuário selecionar o formato de saída
formato_output = widgets.Dropdown(
    options=['texto', 'gráfico', 'mapa_mental'],
    value='texto',
    description='Formato de saída:'
)

# Cria um botão para iniciar a pesquisa
botao_pesquisar = widgets.Button(description='Pesquisar')
output = widgets.Output() # Cria um container para exibir a saída

In [6]:
# Define a função que será executada quando o botão for clicado
def on_botao_pesquisar_clicked(b):
    with output: # Executa as operações dentro do container de saída
        output.clear_output() # Limpa a saída anterior
        resultado = pesquisar_e_resumir(texto_input.value, formato_output.value) # Chama a função pesquisar_e_resumir com o texto inserido pelo usuário e o formato de saída selecionado
        if formato_output.value == 'texto': # Se o formato de saída for texto
            display(Markdown(resultado)) # Exibe o texto formatado

In [7]:
# Define a função on_botao_pesquisar_clicked como a função a ser executada quando o botão for clicado
botao_pesquisar.on_click(on_botao_pesquisar_clicked)

# Exibe a interface do usuário
display(widgets.VBox([texto_input, formato_output, botao_pesquisar, output]))